In [1]:
import time
import retro
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import cv2
#retro.data.list_games()

In [2]:
#obs, rew, done, info = env.step(env.action_space.sample())
#(224, 240, 3)
#MultiBinary(9)
#["B", null, "SELECT", "START", "UP", "DOWN", "LEFT", "RIGHT", "A"]
action_space = [[0,0,0,0,0,0,0,0,0],
                [1,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,1,0,0],
                [0,0,0,0,0,0,0,1,0],
                [0,0,0,0,0,0,0,0,1],
                [1,0,0,0,0,0,1,0,0],
                [1,0,0,0,0,0,0,1,0],
                [1,0,0,0,0,0,0,0,1],
                [0,0,0,0,0,0,1,1,0],
                [0,0,0,0,0,0,1,0,1],
                [0,0,0,0,0,0,0,1,1],
                [1,0,0,0,0,0,0,1,1],
                [1,0,0,0,0,0,1,0,1],
                [1,0,0,0,0,0,1,1,1],
                [0,0,0,0,0,1,0,0,0],
                [1,0,0,0,0,1,0,0,0],
                [0,0,0,0,0,1,0,0,1],
                [0,0,0,0,0,0,1,1,0]]


env = retro.make(game='SuperMarioBros-Nes')

In [78]:
class ANN():
  def __init__(self):
    num_actions = 18
    #NN layers
    image_input = layers.Input(shape=(4,200,200))
    preprocessor = layers.experimental.preprocessing.Resizing(200, 200, interpolation='bilinear', name=None)(image_input)
    # Convolutions on the frames on the screen
    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(preprocessor)
    layer2 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer2)
    layer4 = layers.Flatten()(layer3)
    layer5 = layers.Dense(512, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)

    #Define NN parameters.
    self.toymodel = keras.Model(inputs=image_input, outputs=action)
    self.loss_fn = tf.keras.losses.Huber()
    self.optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)
    self.toymodel.compile(self.optimizer, self.loss_fn)

  def trainStep(self, sample_X, sample_Y):
    with tf.GradientTape() as tape:
      old_q = self.toymodel(sample_X, training=True)
      loss_value = self.loss_fn(sample_Y, old_q)
    grads = tape.gradient(loss_value, self.toymodel.trainable_weights)
    self.optimizer.apply_gradients(zip(grads, self.toymodel.trainable_weights))
    return loss_value.numpy()

  def train(self, x_input, y_input, batchsize=64):
    loss_history = []
    dataset = tf.data.Dataset.from_tensor_slices((x_input, y_input))
    dataset = dataset.shuffle(buffer_size=1024).batch(batchsize)
    for steps, (x, y) in enumerate(dataset):
      loss_history.append(self.trainStep(x,y))
    return loss_history

  def forward(self, x_input):
    return self.toymodel(x_input)

def preprocess(image):
    output = np.average(np.array(image), axis=2)[25:205]
    #return output
    return cv2.resize(output, dsize=(200, 200), interpolation=cv2.INTER_CUBIC)

def popback(state_block, incoming_state):
    state_block.pop(0)
    state_block.append(incoming_state)
    return state_block

def gradient_update(state_history, 
                    next_state_history,
                    rewards_history,
                    action_history,
                    loss_history,
                    model, 
                    gamma,
                    batch_size):
    
            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)
            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            next_state_sample = np.array([next_state_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices])
            print ('Memory contains ', len(action_history), 'states.')
            future_rewards = model.toymodel.predict((np.array(next_state_sample)))
            updated_q_values = rewards_sample + gamma * tf.reduce_max(future_rewards, axis=1)
            masks = tf.one_hot(action_sample, 18)
            with tf.GradientTape() as tape:  
                q_values = model.forward((np.array(state_sample)))
                q_actions = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                loss = model.loss_fn(updated_q_values, q_actions)
            loss_history.append(loss)
            grads = tape.gradient(loss, model.toymodel.trainable_variables)
            print ('Gradient updated. Loss at',float(loss))
            model.toymodel.optimizer.apply_gradients(zip(grads, model.toymodel.trainable_variables))
            model.toymodel.save('120222_SMB')
                                         
def memory_manager(array, memory_length):
    excess = len(array)-memory_length
    

In [79]:
behavior = ANN()
target = ANN()

In [80]:
epsilon = 0
min_epsilon = 0
gamma = 0.99
max_memory_len = 100000
batch_size = 32

In [81]:
loss_history = []
action_history = []
state_history= []
next_state_history = []
reward_history = []
done_history = []
episodic_return = []

In [82]:
for episodes in range(1000):
    s = []
    s.append(preprocess(env.reset()))
    #Prime the state s with 3 frames.
    prelim_reward = 0
    epi_return = 0 
    for i in range(3):
        frame, reward, done, info = env.step(action_space[0])
        prelim_reward += reward
        s.append(preprocess(frame))
    done = False
    #Choose an initial action.
    if np.random.random() < epsilon:
        a = np.random.choice(np.arange(len(action_space)))
    else:
        a = np.argmax(behavior.forward(np.expand_dims(s,0)))
    while not done:
        new_frame, reward, done, info = env.step(action_space[a])
        s_prime = popback(s, preprocess(new_frame))
        epi_return += reward
        #env.render()
        if done:
            #Add gradient minimization step here.
            return_history.append(epi_return+prelim_reward)
            break
        if np.random.random() < epsilon:
            a = np.random.choice(np.arange(len(action_space)))
        else:
            a_prime = np.argmax(behavior.forward(np.expand_dims(s,0)))
        #Save to history
        reward_history.append(prelim_reward)
        state_history.append(s)
        action_history.append(a)
        next_state_history.append(s_prime)
        done_history.append(done)
        if len(reward_history)>32:
            gradient_update(state_history, 
                            next_state_history,
                            reward_history,
                            action_history,
                            loss_history,
                            behavior, 
                            gamma,
                            batch_size)
        s = s_prime
        a = a_prime
        
    plt.figure(figsize=(5,5))
    plt.plot(np.arange(len(episodic_return)), episodic_return)
    plt.ylabel('Return')
    plt.xlabel('Episode')


Memory contains  33 states.
Gradient updated. Loss at 1.36895751953125
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  34 states.
Gradient updated. Loss at 2176.142578125
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  35 states.
Gradient updated. Loss at 108.84371948242188
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  36 states.
Gradient updated. Loss at 16.2894287109375
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  37 states.
Gradient updated. Loss at 11.7626953125
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  38 states.
Gradient updated. Loss at 53.62089538574219
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  39 states.
Gradient updated. Loss at 2.164543628692627
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  40 states.
Gradient updated. Loss at 69.33622741699219
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory c

Memory contains  97 states.
Gradient updated. Loss at 3.037405804207083e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  98 states.
Gradient updated. Loss at 2.670489720912883e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  99 states.
Gradient updated. Loss at 1.5638988770660944e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  100 states.
Gradient updated. Loss at 2.7487067200127058e-05
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  101 states.
Gradient updated. Loss at 7.969680154928938e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  102 states.
Gradient updated. Loss at 8.580049325246364e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  103 states.
Gradient updated. Loss at 2.3342076019616798e-05
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  104 states.
Gradient updated. Loss at 1.1526581147336401e-05
INFO:tensorf

Memory contains  160 states.
Gradient updated. Loss at 2.8230326734046685e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  161 states.
Gradient updated. Loss at 1.3235337519290624e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  162 states.
Gradient updated. Loss at 3.539362069204799e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  163 states.
Gradient updated. Loss at 8.38634696265217e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  164 states.
Gradient updated. Loss at 5.892396529816324e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  165 states.
Gradient updated. Loss at 5.5129421525634825e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  166 states.
Gradient updated. Loss at 1.9288002022221917e-06
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  167 states.
Gradient updated. Loss at 2.954597903226386e-06
INFO:tenso

Memory contains  223 states.
Gradient updated. Loss at 4.349441269368981e-07
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  224 states.
Gradient updated. Loss at 7.203737482086581e-07
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  225 states.
Gradient updated. Loss at 5.675205443367304e-07
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  226 states.
Gradient updated. Loss at 5.370819167183072e-07
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  227 states.
Gradient updated. Loss at 5.794811954729084e-07
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  228 states.
Gradient updated. Loss at 5.700027259081253e-07
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  229 states.
Gradient updated. Loss at 3.962791481626482e-07
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  230 states.
Gradient updated. Loss at 2.9254914579723845e-07
INFO:tensorf

Memory contains  286 states.
Gradient updated. Loss at 1.9415859142668523e-08
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  287 states.
Gradient updated. Loss at 3.397171965957568e-08
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  288 states.
Gradient updated. Loss at 3.0061826805649616e-08
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  289 states.
Gradient updated. Loss at 5.99721801108899e-08
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  290 states.
Gradient updated. Loss at 4.398737729616187e-08
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  291 states.
Gradient updated. Loss at 2.7465258511938373e-08
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  292 states.
Gradient updated. Loss at 3.1549262757835095e-08
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  293 states.
Gradient updated. Loss at 1.6946170688925122e-08
INFO:tens

...

INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2690 states.
Gradient updated. Loss at 34.29383850097656
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2691 states.
Gradient updated. Loss at 18.589141845703125
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2692 states.
Gradient updated. Loss at 23.148845672607422
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2693 states.
Gradient updated. Loss at 24.83228874206543
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2694 states.
Gradient updated. Loss at 14.055448532104492
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2695 states.
Gradient updated. Loss at 5.878694534301758
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2696 states.
Gradient updated. Loss at 2.9631333351135254
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2697 states.
Gradient updated. Loss a

Memory contains  2754 states.
Gradient updated. Loss at 0.1092335432767868
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2755 states.
Gradient updated. Loss at 0.09586602449417114
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2756 states.
Gradient updated. Loss at 0.05630180984735489
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2757 states.
Gradient updated. Loss at 0.15180179476737976
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2758 states.
Gradient updated. Loss at 0.0926714539527893
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2759 states.
Gradient updated. Loss at 0.12352340668439865
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2760 states.
Gradient updated. Loss at 0.1398204267024994
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2761 states.
Gradient updated. Loss at 0.12773080170154572
INFO:tensorflow:Assets w

Memory contains  2818 states.
Gradient updated. Loss at 29.291603088378906
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2819 states.
Gradient updated. Loss at 29.027631759643555
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2820 states.
Gradient updated. Loss at 34.89479064941406
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2821 states.
Gradient updated. Loss at 37.74991989135742
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2822 states.
Gradient updated. Loss at 27.231922149658203
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2823 states.
Gradient updated. Loss at 25.419422149658203
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2824 states.
Gradient updated. Loss at 34.28924560546875
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2825 states.
Gradient updated. Loss at 34.608943939208984
INFO:tensorflow:Assets written t

INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2883 states.
Gradient updated. Loss at 5.225822448730469
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2884 states.
Gradient updated. Loss at 0.8980261087417603
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2885 states.
Gradient updated. Loss at 3.0594773292541504
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2886 states.
Gradient updated. Loss at 1.0244895219802856
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2887 states.
Gradient updated. Loss at 0.8609308004379272
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2888 states.
Gradient updated. Loss at 0.7624961137771606
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2889 states.
Gradient updated. Loss at 0.779430627822876
INFO:tensorflow:Assets written to: 120222_SMB\assets
Memory contains  2890 states.
Gradient updated. Loss 

KeyboardInterrupt: 

In [1]:
env.render()

NameError: name 'env' is not defined